## Single-way wave solution

### Using package

In [1]:
using Pkg; Pkg.activate("/work/b11209013/external/JuliaENV/atmo");

using HDF5
using LinearAlgebra, Statistics

  Activating project at `/work/b11209013/external/JuliaENV/atmo`


### Setting Parameters

In [2]:
struct ModelParams
    b1::Float64 
    b2::Float64    
    c1::Float64 
    c2::Float64
    m1::Float64 
    m2::Float64
    γ0::Float64 
    γq::Float64
    F ::Float64 
    f ::Float64
    τL::Float64 
    ϵ ::Float64
end

param = ModelParams(1.0, 2.0, 1.0, 0.5, 0.3, 1.0, 0.0, 0.7, 4.0, 0.5, 1/12, 0.1)

ModelParams(1.0, 2.0, 1.0, 0.5, 0.3, 1.0, 0.0, 0.7, 4.0, 0.5, 0.08333333333333333, 0.1)

### Functions

In [3]:
# coefficient matrix for dynamical system
function coeff_matrix(
        kn::Float64;
        ϵ=param.ϵ  , b1=param.b1, b2=param.b2,
        c1=param.c1, c2=param.c2, γq=param.γq,
        γ0=param.γ0, m1=param.m1, m2=param.m2,
        τL=param.τL, f=param.f  , F =param.F ,
)
    α ::ComplexF64 = -1.5*b2*γq - F*f*(im*kn*c1-ϵ) - 1.5*F*(1-f)*γq;
    β ::ComplexF64 = -F*(1-f)*(im*kn*c2-ϵ);
    γ ::ComplexF64 = (b2+F*(1-f))*γq;
    δ ::ComplexF64 = -F*f - F*(1-f)*γ0 - (b1+b2*γ0);
    
    relax :: ComplexF64 = τL*(b1+b2*γ0);

    mat :: Array{ComplexF64}= [
        (im*kn*c1-ϵ) 0.0         0.0     1.0     ;
        1.5*γq       (im*kn*c2-ϵ) -γq     γ0      ;
        1.5*m2*γq    0.0         -m2*γq  m1+m2*γ0;
        α/relax      β/relax     γ/relax δ/relax ; 
    ]

    return mat
end

function modal(kn::Float64)
    mat :: Array{ComplexF64} = coeff_matrix(kn);

    growth_rate :: Array{Float64} = real.(eigvals(mat))

    return maximum(growth_rate)
end

function nonmodal(kn::Float64)
    mat :: Matrix{ComplexF64} = coeff_matrix(kn);

    unit_vec :: Matrix{ComplexF64} = ones(ComplexF64, 4,1);
    sys_mat  :: Matrix{ComplexF64} = exp(mat);
    pred_vec :: Matrix{ComplexF64} = exp(mat)*unit_vec;

    amp_factor :: Matrix{ComplexF64} = (pred_vec'*pred_vec) * inv(unit_vec'*unit_vec)

    return real.(amp_factor[1,1])
end

nonmodal (generic function with 1 method)

### Linear stability test

In [4]:
λ :: Array{Float64} = collect(540:540:43200)
k :: Array{Float64} = @. 2*π*4320.0 / λ;

modal_stab    :: Array{Float64} = modal.(k);
nonmodal_stab :: Array{Float64} = nonmodal.(k);

### Save the HDF5 file

In [5]:
h5open("/home/b11209013/2025_Research/MSI/File/oneway_stability.h5", "w") do f
    write(f, "nonmodal", nonmodal_stab)
    write(f, "modal", modal_stab)
    write(f, "kn", k)

    attributes(f["nonmodal"])["standard_name"] = "nonmodal growth rate"
    attributes(f["modal"])["standard_name"] = "modal growth rate"
    attributes(f["modal"])["units"] = "1/day"
    attributes(f["kn"])["standard_name"] = "angular wavenumber"
    attributes(f["kn"])["units"] = "None"
end

"None"